In [1]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

In [2]:
# Define paths
base_dir = 'train'

In [3]:
# setup for data generators
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.1,
    rotation_range=10,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

train_gen = datagen.flow_from_directory(
    base_dir,
    target_size=(256, 256),
    batch_size=16,
    class_mode='binary',
    subset='training'
)

valid_gen = datagen.flow_from_directory(
    base_dir,
    target_size=(256, 256),
    batch_size=16,
    class_mode='binary',
    subset='validation'
)


Found 2374 images belonging to 2 classes.
Found 263 images belonging to 2 classes.


In [4]:
# model functions
from tensorflow.keras.applications import MobileNetV2, ResNet50, VGG16

def build_model(base_model):
    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dense(64, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# MobileNetV2 Model
base_mnv2 = MobileNetV2(weights='imagenet', include_top=False, input_shape=(256, 256, 3))
base_mnv2.trainable = False
model_mnv2 = build_model(base_mnv2)

# ResNet50 Model
base_resnet = ResNet50(weights='imagenet', include_top=False, input_shape=(256, 256, 3))
base_resnet.trainable = False
model_resnet = build_model(base_resnet)

# VGG16 Model
base_vgg = VGG16(weights='imagenet', include_top=False, input_shape=(256, 256, 3))
base_vgg.trainable = False
model_vgg = build_model(base_vgg)


C:\Users\nandi\AppData\Local\Temp\ipykernel_23600\1027193824.py:15: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_mnv2 = MobileNetV2(weights='imagenet', include_top=False, input_shape=(256, 256, 3))


In [35]:
model_mnv2.save('skin_cancer_mnv2.h5')
model_resnet.save('skin_cancer_resnet50.h5')
model_vgg.save('skin_cancer_vgg16.h5')


In [5]:
#Training the modes

# Train MobileNetV2
print("Training MobileNetV2")
base_mnv2.trainable = True 
model_mnv2.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
history_mnv2 = model_mnv2.fit(
    train_gen,
    validation_data=valid_gen,
    epochs=5  
)


# Train ResNet50
print("Training ResNet50")
base_resnet.trainable = True
model_resnet.compile(optimizer=Adam(learning_rate=0.0001), 
                     loss='binary_crossentropy', 
                     metrics=['accuracy'])
history_resnet = model_resnet.fit(
    train_gen,
    validation_data=valid_gen,
    epochs=5
)

# Train VGG16
print("Training VGG16")
base_vgg.trainable = True
model_vgg.compile(optimizer=Adam(learning_rate=0.0001), 
                  loss='binary_crossentropy', 
                  metrics=['accuracy'])
history_vgg = model_vgg.fit(
    train_gen,
    validation_data=valid_gen,
    epochs=5
)


Training MobileNetV2


C:\Users\nandi\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
149/149 ━━━━━━━━━━━━━━━━━━━━ 383s 2s/step - accuracy: 0.7475 - loss: 0.4971 - val_accuracy: 0.5475 - val_loss: 1.1225
Epoch 2/5
149/149 ━━━━━━━━━━━━━━━━━━━━ 343s 2s/step - accuracy: 0.8710 - loss: 0.2819 - val_accuracy: 0.5475 - val_loss: 1.2827
Epoch 3/5
149/149 ━━━━━━━━━━━━━━━━━━━━ 313s 2s/step - accuracy: 0.8868 - loss: 0.2403 - val_accuracy: 0.5513 - val_loss: 1.7992
Epoch 4/5
149/149 ━━━━━━━━━━━━━━━━━━━━ 325s 2s/step - accuracy: 0.9098 - loss: 0.1968 - val_accuracy: 0.6046 - val_loss: 0.8210
Epoch 5/5
149/149 ━━━━━━━━━━━━━━━━━━━━ 308s 2s/step - accuracy: 0.9268 - loss: 0.1852 - val_accuracy: 0.7338 - val_loss: 0.6257
Training ResNet50
Epoch 1/5
149/149 ━━━━━━━━━━━━━━━━━━━━ 1096s 7s/step - accuracy: 0.7974 - loss: 0.4113 - val_accuracy: 0.5475 - val_loss: 0.8015
Epoch 2/5
149/149 ━━━━━━━━━━━━━━━━━━━━ 737s 5s/step - accuracy: 0.8797 - loss: 0.2553 - val_accuracy: 0.5475 - val_loss: 0.8548
Epoch 3/5
149/149 ━━━━━━━━━━━━━━━━━━━━ 738s 5s/step - accuracy: 0.9044 - loss: 0.2053

In [ ]:
# Combine predictions using a simple (unweighted) average ensemble
ensemble_preds = (preds_mnv2 + preds_resnet + preds_vgg) / 3.0

# Convert probabilities to binary predictions using a threshold of 0.5
ensemble_binary = (ensemble_preds > 0.5).astype(int)

# Retrieve the true labels from the test generator
true_labels = test_gen.labels

# Compute ensemble accuracy
ensemble_accuracy = accuracy_score(true_labels, ensemble_binary)
print("Ensemble Test Accuracy (Simple Average):", ensemble_accuracy)

# Optionally, inspect a few sample ensemble predictions (probabilities)
print("Sample Ensemble Predictions (first 5):", ensemble_preds[:10])

In [ ]:
model_mnv2.save('skin_cancer_mnv2.h5')
model_resnet.save('skin_cancer_resnet50.h5')
model_vgg.save('skin_cancer_vgg16.h5')


TESTING

In [25]:
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score

# Define the test directory path
test_dir = 'test'  

# Create a test ImageDataGenerator that only rescales the images
test_datagen = ImageDataGenerator(rescale=1./255)

# Load test data from directory
test_gen = test_datagen.flow_from_directory(
    test_dir,
    target_size=(256, 256),  # Use the same target size as during training
    batch_size=16,
    class_mode='binary',
    shuffle=False  # Important: keep order for accurate label matching
)

# Get predictions from each trained model on the test set
preds_mnv2 = model_mnv2.predict(test_gen)
preds_resnet = model_resnet.predict(test_gen)
preds_vgg = model_vgg.predict(test_gen)


Found 660 images belonging to 2 classes.


C:\Users\nandi\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


42/42 ━━━━━━━━━━━━━━━━━━━━ 24s 528ms/step
42/42 ━━━━━━━━━━━━━━━━━━━━ 69s 2s/step
42/42 ━━━━━━━━━━━━━━━━━━━━ 189s 4s/step
